In [ ]:
#loading model and running reservoir data through the model
#Jessica Besnier

In [30]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.tree import DecisionTreeRegressor
import joblib



# Load the trained pipeline
loaded_pipeline = joblib.load('Z:/Research_Projects/Reservoir_Project/ML_outputs/models/Run_test_RandomForestRegressor_35_model.joblib')

# Prepare new data
res_name = 'agua_vermelha'
input_data_path = 'Z:/Research_Projects/Reservoir_Project/data/regression_variables/varaibles_ts_clean_top_anom/' + res_name + '_var_normalized.csv'

# Load input data
input_data = pd.read_csv(input_data_path)
input_data = input_data.dropna()

def create_regression_dataset():
    df = input_data[['normalized_reservoir_height', 'tws', 'precipitation', 'temp', 'volume', 'area', 'discharge', 'Res_time', 'depth', 'wshd_area','elevation' ]] #timeseries and static varaibles for algortihm to tell the difference between reservoirs 
    
    X = df[['tws', 'precipitation', 'temp', 'volume', 'area', 'discharge', 'Res_time', 'depth', 'wshd_area','elevation']]
    y = df['normalized_reservoir_height']
    return df, X, y

def create_datasets():
    """Create example datasets"""

    df_regression, _, __ = create_regression_dataset()
    return df_regression, 

df_regression = create_datasets()


In [31]:

def time_series_to_tabular(included_columns):
    df, _, __ = create_regression_dataset()

    TARGET = 'normalized_reservoir_height' # The column in df we want to forecast
    LAG = 36 # This is how far back we want to look for features
    HORIZON = 6 # This is how far forward we want forecast

    # Fill in missing values
    cols = df.columns
    index = df.index
    df = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(df)
    df = pd.DataFrame(df, columns=cols, index=index) # convert back to dataframe

    df = df[included_columns]    


    def create_lag_features(df, target, lag):
        
        for col in df.columns:
            for i in range(1, lag+1):
                df[f'{col}-{i}'] = df[col].shift(i)

            # Drop non-target values (we only keep historical feature values)
            if col != target:
                df = df.drop(col, axis=1)
        # OPTIONAL: Drop first N rows where N = lag
        # Alternatively, we could impute the missing data
        df = df.iloc[lag:]
        return df 

    def create_future_values(df, target, horizon):
        targets = [ target ]
        for i in range(1, horizon):
            col_name = f'{target}+{i}'
            df[col_name] = df[target].shift(-i)
            targets.append(col_name)

        # Optional: Drop rows missing future target values
        df = df[df[targets[-1]].notna()]
        return df, targets


    print('\nInitial df shape:', df.shape)
    # Create feature data (X)
    df = create_lag_features(df, TARGET, LAG)
    print('\ndf lag shape after feature creation:', df.shape)

    # Create targets to forecast (y)
    df, targets = create_future_values(df, TARGET, HORIZON)
    print('\ndf horizon shape after feature creation:', df.shape)

    # Separate features (X) and targets (y)
    y = df[targets]
    X = df.drop(targets, axis=1)
    #df.set_index('time', inplace=True)

    print('\nShape of X (features):', X.shape)
    print('Shape of y (target(s)):', y.shape)

    return X, y

In [32]:
if __name__ == '__main__':

    time_series_to_tabular(included_columns= ['normalized_reservoir_height', 'tws', 'precipitation', 'temp',])


Initial df shape: (167, 4)

df lag shape after feature creation: (131, 145)

df horizon shape after feature creation: (126, 150)

Shape of X (features): (126, 144)
Shape of y (target(s)): (126, 6)


In [35]:


X, y = time_series_to_tabular(included_columns= ['normalized_reservoir_height', 'tws', 'precipitation', 'temp',])
# #through pipeline
scaler = loaded_pipeline.named_steps['scaler']
X_new_normalized = scaler.transform(X)
predictions_new = loaded_pipeline.predict(X_new_normalized)

#predictions_new = loaded_pipeline.predict(X)
#another option
# print(loaded_pipeline.named_steps)
# multioutput_regressor = loaded_pipeline.named_steps['multioutput']
# model_regressor = multioutput_regressor.estimators_[0]  
# X_new_transformed = X_new

# predictions_new = model_regressor.predict(X_new_transformed)



Initial df shape: (167, 4)

df lag shape after feature creation: (131, 145)

df horizon shape after feature creation: (126, 150)

Shape of X (features): (126, 144)
Shape of y (target(s)): (126, 6)


c:\Users\jess_besnier\anaconda3\envs\mlenv\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but Normalizer was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: X has 144 features, but Normalizer is expecting 35 features as input.